# Lab 1: Great Expectations Basics - Giới thiệu Data Quality

## 🎯 Objectives
- Hiểu Great Expectations là gì và tại sao sử dụng
- Cài đặt và cấu hình Great Expectations
- Hiểu Data Context và Data Sources
- Tạo Expectations đầu tiên
- Validate data với Expectations

## 📋 Prerequisites
- Python 3.10+ đã cài đặt
- PostgreSQL đang chạy (Docker Compose)
- Conda environment đã setup

## 🏗️ Great Expectations Overview

**Great Expectations (GE)** là công cụ cho data quality:
- **Expectations**: Declarative statements về data
- **Validation**: Check data có match expectations không
- **Data Docs**: Tự động generate documentation
- **Checkpoints**: Automated validation workflows

### GE Architecture:
```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Data      │────▶│ Expectations │────▶│ Validation  │
│  Sources    │     │   (Rules)    │     │  Results    │
└─────────────┘     └─────────────┘     └─────────────┘
                            │
                            ▼
                    ┌─────────────┐
                    │  Data Docs  │
                    │ (Auto-gen)  │
                    └─────────────┘
```


## 1. Kiểm tra Installation


In [ ]:
# Kiểm tra Great Expectations version
import subprocess
import sys

def check_ge_installation():
    """Kiểm tra Great Expectations đã được cài đặt chưa"""
    try:
        result = subprocess.run(
            ['great_expectations', '--version'],
            capture_output=True,
            text=True,
            check=True
        )
        print("✅ Great Expectations đã được cài đặt:")
        print(result.stdout)
        return True
    except FileNotFoundError:
        print("❌ Great Expectations chưa được cài đặt")
        print("💡 Chạy: pip install great-expectations")
        return False
    except subprocess.CalledProcessError as e:
        print(f"⚠️  Lỗi khi kiểm tra GE: {e}")
        return False

check_ge_installation()


## 2. Import Great Expectations

Import và setup Great Expectations.


In [ ]:
# Import Great Expectations
import great_expectations as gx
from great_expectations.core import ExpectationSuite
from great_expectations.datasource.fluent import Datasource
import pandas as pd
from pathlib import Path

print("✅ Great Expectations imported successfully!")
print(f"GE Version: {gx.__version__}")

# Get project root
project_root = Path('.').resolve()
if project_root.name != 'Great_Expectations_lab':
    project_root = project_root.parent / 'Great_Expectations_lab'

print(f"\n📁 Project Root: {project_root}")


## 3. Initialize Great Expectations Context

Tạo Data Context - trung tâm của GE project.


In [ ]:
# Initialize Great Expectations Context
import os

# Change to project directory
original_dir = os.getcwd()
try:
    os.chdir(project_root)
    
    # Initialize context (or load existing)
    try:
        context = gx.get_context()
        print("✅ Loaded existing Great Expectations context")
    except Exception:
        print("⚠️  No existing context found")
        print("💡 Run: great_expectations init --no-view")
        print("   Or initialize programmatically below")
        
        # Initialize programmatically
        context = gx.get_context(mode="file")
        print("✅ Created new Great Expectations context")
    
    print(f"\n📁 Context Root Directory: {context.root_directory}")
    print(f"📁 Data Docs Directory: {context.get_docs_sites()}")
    
except Exception as e:
    print(f"⚠️  Error: {e}")
finally:
    os.chdir(original_dir)


## 4. Tạo Data Source

Kết nối với data source (PostgreSQL hoặc Pandas DataFrame).


In [ ]:
# Example: Tạo datasource với Pandas DataFrame
sample_data = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5],
    'first_name': ['John', 'Jane', 'Bob', 'Alice', 'Charlie'],
    'email': ['john@email.com', 'jane@email.com', 'bob@email.com', 'alice@email.com', 'charlie@email.com'],
    'age': [30, 25, 35, 28, 32],
    'total_orders': [5, 3, 8, 2, 6],
    'lifetime_value': [1500.00, 800.00, 2500.00, 600.00, 1800.00]
})

print("📊 Sample Data:")
print(sample_data)
print(f"\nShape: {sample_data.shape}")
print(f"Columns: {list(sample_data.columns)}")

# Create Data Asset từ DataFrame
try:
    os.chdir(project_root)
    context = gx.get_context()
    
    # Tạo datasource với Pandas
    datasource = context.sources.add_pandas(name="pandas_datasource")
    data_asset = datasource.add_dataframe_asset(name="customers_df")
    
    print("\n✅ Created Pandas datasource và data asset")
    print(f"Datasource: {datasource.name}")
    print(f"Data Asset: {data_asset.name}")
    
except Exception as e:
    print(f"⚠️  Error creating datasource: {e}")
    print("💡 Make sure GE context is initialized")
finally:
    os.chdir(original_dir)


## 5. Tạo Expectations đầu tiên

Tạo Expectations để validate data quality.


In [ ]:
# Tạo Expectations với Pandas DataFrame
validator = gx.from_pandas(sample_data)

print("🔮 Creating Expectations...")
print("=" * 60)

# Column-level expectations
validator.expect_column_to_exist("customer_id")
validator.expect_column_values_to_not_be_null("customer_id")
validator.expect_column_values_to_be_unique("customer_id")

validator.expect_column_values_to_not_be_null("email")
validator.expect_column_values_to_match_regex("email", r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$")

validator.expect_column_values_to_be_between("age", min_value=18, max_value=100)
validator.expect_column_values_to_be_between("total_orders", min_value=0)

# Table-level expectations
validator.expect_table_row_count_to_be_between(min_value=1, max_value=1000)
validator.expect_table_columns_to_match_ordered_list([
    "customer_id", "first_name", "email", "age", "total_orders", "lifetime_value"
])

print("✅ Created Expectations:")
for expectation in validator.get_expectation_suite().expectations:
    print(f"  - {expectation.expectation_type}")

print("\n💡 Expectations được lưu trong Expectation Suite")
print(f"Suite Name: {validator.get_expectation_suite().expectation_suite_name}")


## 6. Validate Data

Chạy validation để kiểm tra data có match expectations không.


In [ ]:
# Validate data
validation_result = validator.validate()

print("🧪 Validation Results:")
print("=" * 60)
print(f"Success: {validation_result.success}")
print(f"Statistics:")
print(f"  - Evaluated Expectations: {validation_result.statistics['evaluated_expectations']}")
print(f"  - Successful Expectations: {validation_result.statistics['successful_expectations']}")
print(f"  - Unsuccessful Expectations: {validation_result.statistics['unsuccessful_expectations']}")
print(f"  - Success Percent: {validation_result.statistics['success_percent']:.2f}%")

print("\n📋 Results Details:")
for result in validation_result.results:
    status = "✅" if result.success else "❌"
    print(f"{status} {result.expectation_config.expectation_type}: {result.success}")

if not validation_result.success:
    print("\n⚠️  Some expectations failed!")
    print("Check results above for details")
else:
    print("\n✅ All expectations passed!")


## 7. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Great Expectations là gì và tại sao sử dụng
2. Cài đặt và import GE
3. Initialize Data Context
4. Tạo Data Sources
5. Tạo Expectations đầu tiên
6. Validate data với Expectations

### 📚 Next Lab:
- **Lab 2**: Expectations
- Các loại Expectations
- Column-level và table-level expectations
- Custom expectations
- Expectation suites

### 💡 Key Takeaways:

**Great Expectations:**
- Data quality và validation tool
- Declarative expectations
- Automated validation
- Auto-generated documentation

**Key Concepts:**
- Data Context: Trung tâm của GE project
- Data Sources: Nguồn data (Pandas, SQL, etc.)
- Expectations: Rules về data quality
- Validation: Check data match expectations

### 🔗 Useful Links:
- [Great Expectations Documentation](https://docs.greatexpectations.io/)
- [GE GitHub](https://github.com/great-expectations/great_expectations)
- [GE Expectations Gallery](https://greatexpectations.io/expectations/)
